In [1]:
import pandas as pd
import numpy as np
from functools import reduce
import matplotlib.pyplot as plt
import seaborn as sns
import time

%matplotlib inline
pd.set_option('display.max_columns', None)
pd.options.mode.chained_assignment = None  # To ignore SettingWithCopyWarning warning

# Import data

In [2]:
path = 'C:/Users/Akis/OneDrive/Masters/Web Economics/Data/WebEcon Team Drive/Datasets Original/'
train = pd.read_csv(path+"train.csv")
validation = pd.read_csv(path+"validation.csv")

In [3]:
print(train.shape, validation.shape)

(2430981, 25) (303925, 25)


# Import Predictions

In [4]:
path='C:/Users/Akis/OneDrive/Masters/Web Economics/Data/WebEcon Team Drive/predictions/'
# lr_preds = pd.read_csv("Prediction_csv/lr_pCTR.csv", index_col  = "Unnamed: 0")
# rf_preds = pd.read_csv("Prediction_csv/rf_pCTR.csv", index_col  = "Unnamed: 0")
####xgb_preds = pd.read_csv("Prediction_csv/xgb_pCTR.csv", index_col  = "Unnamed: 0")
# dwane_preds = pd.read_csv("Prediction_csv/click_predictions_narrow.csv")
# lasso_preds = pd.read_csv("Prediction_csv/lasso_basebid.csv", index_col  = "Unnamed: 0")
# ridge_preds = pd.read_csv("Prediction_csv/ridge_basebid.csv", index_col  = "Unnamed: 0")
# elasticnet_preds = pd.read_csv("Prediction_csv/elasticnet_basebid.csv", index_col  = "Unnamed: 0")
# rfg_preds = pd.read_csv("Prediction_csv/rfg_basebid.csv", index_col  = "Unnamed: 0")
####xgbr_preds = pd.read_csv("Prediction_csv/xgb_basebid.csv", index_col  = "Unnamed: 0")

xgb_preds = pd.read_csv(path+"xgb_pCTR.csv", index_col  = "Unnamed: 0")
xgbr_preds = pd.read_csv(path+"xgb_basebid.csv", index_col  = "Unnamed: 0")

In [5]:
xgbr_preds.head()

,bidid,predicted_payprice
0,bbcb813b6166538503d8b33a5602d7d72f6019dc,50.281219
1,5a07316c49477cb5d9b4d5aa39c27d6c3be7f92d,78.790169
2,f6ece71dae81d6b16bfb24ad6dd5611472d4c673,92.860153
3,b4d5c57c9b38ff5a12954fa01e11931b4e6bfbbb,45.761795
4,0899bf144249458ea9c89188473694bf44c7ca15,64.029907


In [6]:
print("Predicted Bid Price Min:",xgbr_preds['predicted_payprice'].min())
print("Predicted Bid Price Max:",xgbr_preds['predicted_payprice'].max())
print("Predicted Bid Price Mean:",xgbr_preds['predicted_payprice'].mean())
print("Predicted Bid Price Std:",xgbr_preds['predicted_payprice'].std())

Predicted Bid Price Min: -56.3769416809082
Predicted Bid Price Max: 346.9530944824219
Predicted Bid Price Mean: 78.97979187519267
Predicted Bid Price Std: 36.59964614741393


# Linear Strategy

In [7]:
def Strategy1(sub_pCTR, sub_basebid, C ,BBimportance):
    
    #print(sub_basebid.head(100))
    importance = BBimportance * 10 
    avgCTR = sub_pCTR["click_proba"].mean()
    
    mean = sub_basebid["predicted_payprice"].mean()
    sub_basebid["predicted_payprice"] = sub_basebid["predicted_payprice"]-mean
    sub_basebid["predicted_payprice"] = (importance * sub_basebid["predicted_payprice"]/sub_basebid["predicted_payprice"].max())
    sub_basebid["predicted_payprice"] = sub_basebid["predicted_payprice"] + mean
    #importance is a constant to be played around with
    
    avgBB = sub_basebid["predicted_payprice"].mean()
    
    print("avBB/pBB Min:",(avgBB/sub_basebid["predicted_payprice"]).min())
    print("avBB/pBB Max:",(avgBB/sub_basebid["predicted_payprice"]).max())
#     print("avBB/pBB Mean:",(avgBB/sub_basebid["predicted_payprice"]).mean())
#     print("avBB/pBB Std:",(avgBB/sub_basebid["predicted_payprice"]).std())
#     print()
    
#     print("pCTR/avCTR Min:",(sub_pCTR["click_proba"]/ avgCTR).min())
#     print("pCTR/avCTR Max:",(sub_pCTR["click_proba"]/ avgCTR).max())
#     print("pCTR/avCTR Mean:",(sub_pCTR["click_proba"]/ avgCTR).mean())
#     print("pCTR/avCTR Std:",(sub_pCTR["click_proba"]/ avgCTR).std())
    
    bidprice1 = (avgBB/sub_basebid["predicted_payprice"]) * (sub_pCTR["click_proba"]/ avgCTR) * C
    #print(bidprice1.head(5))
    #sub_pCTR_["click_proba"] = (sub_pCTR_["click_proba"] + (1-avgCTR))**2 - (1-avgCTR)
    #bidprice2 = (avgBB/sub_basebid["predicted_payprice"]) * (sub_pCTR["click_proba"]/ avgCTR) * C
    return bidprice1

In [8]:
def Strategy2(sub_pCTR, sub_basebid, C ,BBimportance):
    
    #print(sub_basebid.head(2))
    importance = BBimportance * 10 
    avgCTR = sub_pCTR["click_proba"].mean()
    
    mean = sub_basebid["predicted_payprice"].mean()
    sub_basebid["predicted_payprice"] = sub_basebid["predicted_payprice"]-mean
    sub_basebid["predicted_payprice"] = (importance * sub_basebid["predicted_payprice"]/sub_basebid["predicted_payprice"].max())
    sub_basebid["predicted_payprice"] = sub_basebid["predicted_payprice"] + mean
    #importance is a constant to be played around with
    
    avgBB = sub_basebid["predicted_payprice"].mean()
    
    print("avBB/pBB Min:",(avgBB/sub_basebid["predicted_payprice"]).min())
    print("avBB/pBB Max:",(avgBB/sub_basebid["predicted_payprice"]).max())
#     print("avBB/pBB Mean:",(avgBB/sub_basebid["predicted_payprice"]).mean())
#     print("avBB/pBB Std:",(avgBB/sub_basebid["predicted_payprice"]).std())
#     print()
    
#     print("pCTR/avCTR Min:",(sub_pCTR["click_proba"]/ avgCTR).min())
#     print("pCTR/avCTR Max:",(sub_pCTR["click_proba"]/ avgCTR).max())
#     print("pCTR/avCTR Mean:",(sub_pCTR["click_proba"]/ avgCTR).mean())
#     print("pCTR/avCTR Std:",(sub_pCTR["click_proba"]/ avgCTR).std())
    
    #bidprice1 = (avgBB/sub_basebid["predicted_payprice"]) * (sub_pCTR["click_proba"]/ avgCTR) * C
    sub_pCTR["click_proba"] = (sub_pCTR["click_proba"] + (1-avgCTR))**2 - (1-avgCTR)
    bidprice2 = (avgBB/sub_basebid["predicted_payprice"]) * (sub_pCTR["click_proba"]/ avgCTR) * C
    return bidprice2

In [9]:
#temp1 = Strategy1(xgb_preds, xgbr_preds, 93, 0) # 1 to 15 is a good range

In [10]:
base_bid = train.loc[ train["click"] == 1, "payprice"].mean()
base_bid

105.46402677077523

In [11]:
base_bid = 89
basebids = np.linspace(0.90*base_bid, 1.06*base_bid, 8)
basebids = np.linspace(base_bid+4, base_bid+10, 7)
basebids

array([93., 94., 95., 96., 97., 98., 99.])

# Evaluate Results

In [12]:
budget = 6250*1000

def ValidationDataFrame(submission_pCTR, submission_basebid, C, BBimportance,strategy):
    
    validation_check = validation[["bidid", "click", "bidprice", "payprice"]]
    validation_check["click_proba"] = submission_pCTR["click_proba"]
    validation_check["bidprice_predicted"] = strategy(submission_pCTR, submission_basebid, C , BBimportance)
    #print(validation_check.head(5))
    #print()
    
    return validation_check


def ValidateStrategy(df):
    
    impressions = 0
    clicks = 0
    cost = 0
    auctions_participated = 0
    balance = budget
    
    for row in df.iterrows():
        
        if cost < budget:
            
            auctions_participated+=1
            
            if (row[1]["bidprice_predicted"] >= row[1]["payprice"]):
                
                if (balance > row[1]["bidprice_predicted"]):
                    
                    impressions+=1
                    clicks+=row[1]["click"]
                    cost+=row[1]["payprice"]
                    balance-=row[1]["payprice"]
                else:
                    pass
    
        else:
            break
    
    # Metrics
    ctr = clicks*100/impressions
    cpm = cost/impressions
    cpc = cost/clicks/1000
    
    print("Strategy statistics:")
    print("Auctions participated: {} | Impressions: {} | Clicks: {} | Cost: {} | CTR: {} | CPM: {} | CPC: {}".format(auctions_participated, \
                                                                                                                     impressions, clicks, \
                                                                                                                     cost,\
                                                                                                                     np.round(ctr,5), \
                                                                                                                     np.round(cpm,5), \
                                                                                                                     np.round(cpc,2)))
    print("\n")
            
    return impressions, clicks, cost, auctions_participated, ctr, cpm, cpc

In [13]:
pCTR_model_name = "XGBoost"
results = pd.DataFrame(columns = ["pCTR_model", "Coefficient", "Impressions", "Clicks", "Cost", "CTR", "CPM", "CPC"])
i=0
Strategies=[Strategy1, Strategy2]

start = time.time()

for C in basebids:
    for BBIMPORTANCE in range(0,10,1): #10
        for strat in Strategies:
            print("pCTR model:",pCTR_model_name,"  Constant:",C,"  Strategy:",strat.__name__,"  pBB Importance:",BBIMPORTANCE)
            xgbr_preds_temp = xgbr_preds.copy(True)
            xgb_preds_temp = xgb_preds.copy(True)
            validation_check = ValidationDataFrame(xgb_preds_temp, xgbr_preds_temp, C, BBIMPORTANCE,strat)
            #print(validation_check.head(10))
            impressions, clicks, cost, auctions_participated, ctr, cpm, cpc = ValidateStrategy(validation_check)
            results.loc[i] = [pCTR_model_name, C, impressions, clicks, cost, ctr, cpm, cpc]
            i+=1
    
end = time.time()
print("Total time: {} mins".format((end-start)/60))

pCTR model: XGBoost   Constant: 93.0   Strategy: Strategy1   pBB Importance: 0
avBB/pBB Min: 1.000000000005442
avBB/pBB Max: 1.000000000005442
Strategy statistics:
Auctions participated: 303925 | Impressions: 117574 | Clicks: 160 | Cost: 6191992 | CTR: 0.13608 | CPM: 52.66464 | CPC: 38.7


pCTR model: XGBoost   Constant: 93.0   Strategy: Strategy2   pBB Importance: 0
avBB/pBB Min: 1.000000000005442
avBB/pBB Max: 1.000000000005442
Strategy statistics:
Auctions participated: 303925 | Impressions: 86314 | Clicks: 160 | Cost: 5962849 | CTR: 0.18537 | CPM: 69.08322 | CPC: 37.27


pCTR model: XGBoost   Constant: 93.0   Strategy: Strategy1   pBB Importance: 1
avBB/pBB Min: 0.8876149315563202
avBB/pBB Max: 1.0683243449944873
Strategy statistics:
Auctions participated: 303925 | Impressions: 117607 | Clicks: 160 | Cost: 6160770 | CTR: 0.13605 | CPM: 52.38438 | CPC: 38.5


pCTR model: XGBoost   Constant: 93.0   Strategy: Strategy2   pBB Importance: 1
avBB/pBB Min: 0.8876149315563202
avBB/pBB Max:

In [14]:
xgbr_preds.head(10)

,bidid,predicted_payprice
0,bbcb813b6166538503d8b33a5602d7d72f6019dc,50.281219
1,5a07316c49477cb5d9b4d5aa39c27d6c3be7f92d,78.790169
2,f6ece71dae81d6b16bfb24ad6dd5611472d4c673,92.860153
3,b4d5c57c9b38ff5a12954fa01e11931b4e6bfbbb,45.761795
4,0899bf144249458ea9c89188473694bf44c7ca15,64.029907
5,f4c6a9a3b1db6da035c7e2a36d4f5e389095beca,28.720341
6,786a2940f225482dc04d455dc7a7fea436b02e03,100.595055
7,17ada861c6ed0616f88312ba0d3d76c0f95b4940,87.275414
8,6abd8ef3eb678b1afe67dc0cff0aa58413fbf29a,46.565254
9,442ffca310e325abc255e8ff97d9812051336acd,37.040314


In [ ]:
xgb_preds.head(10)

In [ ]:
results.to_csv("ValidationSet_Evaluation_Results_Individual_xgb_xcgr.csv")

In [ ]:
results.head()

In [ ]:
results.sort_values(by = ["Clicks",'CTR'], ascending = False)

In [ ]:
plt.figure(figsize = (10,8))

plt.subplot(4,1,1)
plt.plot(basebids, results["Clicks"])
plt.ylabel("Clicks")

plt.subplot(4,1,2)
plt.plot(basebids, results["Impressions"])
plt.ylabel("Impressions")

plt.subplot(4,1,3)
plt.plot(basebids, results["CTR"])
plt.ylabel("CTR")

plt.subplot(4,1,4)
plt.plot(basebids, results["Cost"])
plt.ylabel("Cost")
plt.xlabel("Constant")

plt.tight_layout()